<a href="https://colab.research.google.com/github/MartinJohannessen/text2RDF/blob/main/jointEntityRelationExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version #must be 3.7

Python 3.7.10


In [2]:
!nvidia-smi

Sun Jul 11 17:20:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!pip install transformers==2.6.0
!pip install icecream
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/bert-base-cased
!git clone https://github.com/MartinJohannessen/SPN4RE.git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
fatal: destination path 'bert-base-cased' already exists and is not an empty directory.
fatal: destination path 'SPN4RE' already exists and is not an empty directory.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
cd /content/SPN4RE

/content/SPN4RE


In [10]:
import argparse, os, torch
import random
import numpy as np
from utils.data import build_data
from trainer.trainer import Trainer
from models.setpred4RE import SetPred4RE

from icecream import ic
from transformers import BertTokenizer
from utils.functions import remove_accents  

In [11]:
def str2bool(v):
    return v.lower() in ('true')
def add_argument_group(name):
    arg = parser.add_argument_group(name)
    return arg
def get_args():
    args, unparsed = parser.parse_known_args()
    if len(unparsed) > 1:
        print("Unparsed args: {}".format(unparsed))
    return args, unparsed
def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# Select dataset and change arguments acordingly as instructed in readme.md 

In [ ]:
inference_mode = False

from google.colab import drive
drive.mount('/content/drive')
parser = argparse.ArgumentParser()
data_arg = add_argument_group('Data')


#data_arg.add_argument('--dataset_name', type=str, default="NYT-exact")
#data_arg.add_argument('--train_file', type=str, default="./data/NYT/exact_data/train.json")
#data_arg.add_argument('--valid_file', type=str, default="./data/NYT/exact_data/valid.json")
#data_arg.add_argument('--test_file', type=str, default="./data/NYT/exact_data/test.json")

# data_arg.add_argument('--dataset_name', type=str, default="NYT-partial")
# data_arg.add_argument('--train_file', type=str, default="./data/NYT/casrel_data/new_train.json")
# data_arg.add_argument('--valid_file', type=str, default="./data/NYT/casrel_data/new_valid.json")
# data_arg.add_argument('--test_file', type=str, default="./data/NYT/casrel_data/new_test.json")


data_arg.add_argument('--dataset_name', type=str, default="WebNLG")
data_arg.add_argument('--train_file', type=str, default="./data/WebNLG/clean_WebNLG/new_train.json")
data_arg.add_argument('--valid_file', type=str, default="./data/WebNLG/clean_WebNLG/new_valid.json")
data_arg.add_argument('--test_file', type=str, default="./data/WebNLG/clean_WebNLG/new_test.json")

data_arg.add_argument('--generated_data_directory', type=str, default="/content/drive/MyDrive/a_colab_folder/SPN4RE/WebNLG/")
data_arg.add_argument('--generated_param_directory', type=str, default="/content/drive/MyDrive/a_colab_folder/SPN4RE/WebNLG/Model_parameters/")
data_arg.add_argument('--bert_directory', type=str, default="/content/bert-base-cased")
data_arg.add_argument("--partial", type=str2bool, default=True)
learn_arg = add_argument_group('Learning')
learn_arg.add_argument('--model_name', type=str, default="SPN4RE")
learn_arg.add_argument('--num_generated_triples', type=int, default=10)
learn_arg.add_argument('--num_decoder_layers', type=int, default=4)
learn_arg.add_argument('--matcher', type=str, default="avg", choices=['avg', 'min'])
learn_arg.add_argument('--na_rel_coef', type=float, default=0.25)
learn_arg.add_argument('--rel_loss_weight', type=float, default=1)
learn_arg.add_argument('--head_ent_loss_weight', type=float, default=2)
learn_arg.add_argument('--tail_ent_loss_weight', type=float, default=2)
learn_arg.add_argument('--fix_bert_embeddings', type=str2bool, default=True)
learn_arg.add_argument('--batch_size', type=int, default=4)
learn_arg.add_argument('--max_epoch', type=int, default=100)
learn_arg.add_argument('--gradient_accumulation_steps', type=int, default=1)
learn_arg.add_argument('--decoder_lr', type=float, default=0.00005)
learn_arg.add_argument('--encoder_lr', type=float, default=0.00002)
learn_arg.add_argument('--lr_decay', type=float, default=0.02)
learn_arg.add_argument('--weight_decay', type=float, default=0.000001)
learn_arg.add_argument('--max_grad_norm', type=float, default=20)
learn_arg.add_argument('--optimizer', type=str, default='AdamW', choices=['Adam', 'AdamW'])
evaluation_arg = add_argument_group('Evaluation')
evaluation_arg.add_argument('--n_best_size', type=int, default=100)
evaluation_arg.add_argument('--max_span_length', type=int, default=10) #NYT webNLG 10
misc_arg = add_argument_group('MISC')
misc_arg.add_argument('--refresh', type=str2bool, default=False)
misc_arg.add_argument('--use_gpu', type=str2bool, default=True)
misc_arg.add_argument('--visible_gpu', type=int, default=0)
misc_arg.add_argument('--random_seed', type=int, default=1)

args, unparsed = get_args()
os.environ["CUDA_VISIBLE_DEVICES"] = str(args.visible_gpu)
for arg in vars(args):
    print(arg, ":",  getattr(args, arg))
set_seed(args.random_seed)
tokenizer = BertTokenizer.from_pretrained(args.bert_directory, do_lower_case=False)
data = build_data(args)
model = SetPred4RE(args, data.relational_alphabet.size())
if inference_mode == False:
    trainer = Trainer(model, data, args)
    trainer.train_model()    

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Unparsed args: ['-f', '/root/.local/share/jupyter/runtime/kernel-687b17a7-662e-4c64-8d6e-cdbedfd6c590.json']
dataset_name : WebNLG
train_file : ./data/WebNLG/clean_WebNLG/new_train.json
valid_file : ./data/WebNLG/clean_WebNLG/new_valid.json
test_file : ./data/WebNLG/clean_WebNLG/new_test.json
generated_data_directory : /content/drive/MyDrive/a_colab_folder/SPN4RE/WebNLG/
generated_param_directory : /content/drive/MyDrive/a_colab_folder/SPN4RE/WebNLG/Model_parameters/
bert_directory : /content/bert-base-cased
partial : True
model_name : SPN4RE
num_generated_triples : 10
num_decoder_layers : 4
matcher : avg
na_rel_coef : 0.25
rel_loss_weight : 1
head_ent_loss_weight : 2
tail_ent_loss_weight : 2
fix_bert_embeddings : True
batch_size : 4
max_epoch : 100
gradient_accumulation_steps : 1
decoder_lr : 5e-05
encoder_lr : 2e-05
lr_decay : 0.02
weight_decay : 1e-06
max_

In [17]:
def process_data_for_inference(input_doc):
    sent_list = []
    lines = input_doc.splitlines()
    for i in range(len(lines)):
        token_sent = [tokenizer.cls_token] + tokenizer.tokenize(remove_accents(lines[i])) + [tokenizer.sep_token]
        sent_id = tokenizer.convert_tokens_to_ids(token_sent)
        sent_list.append([i, sent_id])

    sent_size = len(sent_list)
    sent_idx = [ele[0] for ele in sent_list]
    sent_ids = [ele[1] for ele in sent_list]
    sent_lens = list(map(len, sent_ids))
    max_sent_len = max(sent_lens)
    input_ids = torch.zeros((sent_size, max_sent_len), requires_grad=False).long()
    attention_mask = torch.zeros((sent_size, max_sent_len), requires_grad=False, dtype=torch.float32)
    for idx, (seq, seqlen) in enumerate(zip(sent_ids, sent_lens)):
        input_ids[idx, :seqlen] = torch.LongTensor(seq)
        attention_mask[idx, :seqlen] = torch.FloatTensor([1] * seqlen)
    if args.use_gpu:
        input_ids = input_ids.cuda()
        attention_mask = attention_mask.cuda()
    info = {"seq_len": sent_lens, "sent_idx": sent_idx}
    return input_ids, attention_mask, info

In [18]:
def gen_triples(self, input_ids, attention_mask, info):
    with torch.no_grad():
        outputs = self.forward(input_ids, attention_mask)
        pred_triple = generate_triple(outputs, info, self.args, self.num_classes)
    return pred_triple

In [19]:
def resolve_triples(unresolved_triples):
  resolved_triples = []
  for line in unresolved_triples:
    for pred in unresolved_triples[line]:
      relation = data.relational_alphabet.get_instance(pred.pred_rel)
      head = tokenizer.decode(input_ids[line][pred.head_start_index:pred.head_end_index+1])
      tail = tokenizer.decode(input_ids[line][pred.tail_start_index:pred.tail_end_index+1])
      resolved_triples.append([relation, head, tail])
  return resolved_triples

In [20]:
input_doc = ("Oslo is in Norway and Stockholm is in Sweden\n"
            "but is the capital city of Denmark Reykjavik or Copenhagen?")

In [24]:
#inference=True
#%run -i main.py --bert_directory /content/bert-base-cased --model_name SPN4RE --generated_param_directory /content/drive/MyDrive/a_colab_folder/SPN4RE/model_param --generated_data_directory /content/drive/MyDrive/a_colab_folder/SPN4RE --visible_gpu  0 --num_generated_triples 15 --max_grad_norm 2.5 --na_rel_coef 0.25 --max_epoch 100 --max_span_length 10
#PATH = '/content/drive/MyDrive/a_colab_folder/SPN4RE/model_param SPN4RE_NYT-exact_epoch_91_f1_0.9224.model'

dataset_name : NYT-exact
train_file : ./data/NYT/exact_data/train.json
valid_file : ./data/NYT/exact_data/valid.json
test_file : ./data/NYT/exact_data/test.json
generated_data_directory : /content/drive/MyDrive/a_colab_folder/SPN4RE
generated_param_directory : /content/drive/MyDrive/a_colab_folder/SPN4RE/model_param
bert_directory : /content/bert-base-cased
partial : False
model_name : SPN4RE
num_generated_triples : 15
num_decoder_layers : 3
matcher : avg
na_rel_coef : 0.25
rel_loss_weight : 1
head_ent_loss_weight : 2
tail_ent_loss_weight : 2
fix_bert_embeddings : True
batch_size : 8
max_epoch : 100
gradient_accumulation_steps : 1
decoder_lr : 2e-05
encoder_lr : 1e-05
lr_decay : 0.01
weight_decay : 1e-05
max_grad_norm : 2.5
optimizer : AdamW
n_best_size : 100
max_span_length : 10
refresh : False
use_gpu : True
visible_gpu : 0
random_seed : 1
Data setting is loaded from file:  /content/drive/MyDrive/a_colab_folder/SPN4RENYT-exact_SPN4RE_data.pickle
DATA SUMMARY START:
     Relation Alph

In [25]:
model.load_state_dict(torch.load(PATH)['state_dict'])
model.eval()
model.cuda()

input_ids, attention_mask, info = process_data_for_inference(input_doc)
unresolved_triples = model.gen_triples(input_ids, attention_mask, info)
ic(unresolved_triples)
resolved_triples = resolve_triples(unresolved_triples)
ic(resolved_triples)

ic| unresolved_triples: {0: [Pred_Triple(pred_rel=0, rel_prob=0.9999947547912598, head_start_index=4, head_end_index=4, head_start_prob=1.0, head_end_prob=1.0, tail_start_index=1, tail_end_index=1, tail_start_prob=1.0, tail_end_prob=1.0),
                             Pred_Triple(pred_rel=0, rel_prob=0.9990853071212769, head_start_index=9, head_end_index=9, head_start_prob=1.0, head_end_prob=1.0, tail_start_index=6, tail_end_index=6, tail_start_prob=1.0, tail_end_prob=0.9999997615814209),
                             Pred_Triple(pred_rel=5, rel_prob=0.999998927116394, head_start_index=4, head_end_index=4, head_start_prob=1.0, head_end_prob=1.0, tail_start_index=1, tail_end_index=1, tail_start_prob=1.0, tail_end_prob=1.0),
                             Pred_Triple(pred_rel=9, rel_prob=0.9999951124191284, head_start_index=4, head_end_index=4, head_start_prob=1.0, head_end_prob=1.0, tail_start_index=1, tail_end_index=1, tail_start_prob=1.0, tail_end_prob=1.0),
                             P

[['/location/location/contains', 'Norway', 'Oslo'],
 ['/location/location/contains', 'Sweden', 'Stockholm'],
 ['/location/country/administrative_divisions', 'Norway', 'Oslo'],
 ['/location/country/capital', 'Norway', 'Oslo'],
 ['/location/administrative_division/country', 'Oslo', 'Norway'],
 ['/location/country/capital', 'Sweden', 'Stockholm'],
 ['/location/location/contains', 'Denmark', 'Copenhagen'],
 ['/location/location/contains', 'Denmark', 'Reykjavik'],
 ['/location/country/administrative_divisions', 'Denmark', 'Copenhagen'],
 ['/location/country/capital', 'Denmark', 'Copenhagen'],
 ['/location/administrative_division/country', 'Copenhagen', 'Denmark']]

tensor([  101,  1133,  1110,  1103,  2364,  1331,  1104,  5140, 12952,  1377,
         3174, 13160,  1137,  9409,   136,   102], device='cuda:0')

ic| tokenizer.decode(input_ids[line][pred.head_start_index:pred.head_end_index]): ''
ic| tokenizer.decode(input_ids[line][pred.tail_start_index:pred.tail_end_index]): ''


''

ic| resolved_triples: [['/location/location/contains', 'Norway', 'Oslo'],
                       ['/location/location/contains', 'Sweden', 'Stockholm'],
                       ['/location/country/administrative_divisions', 'Norway', 'Oslo'],
                       ['/location/country/capital', 'Norway', 'Oslo'],
                       ['/location/administrative_division/country', 'Oslo', 'Norway'],
                       ['/location/country/capital', 'Sweden', 'Stockholm'],
                       ['/location/location/contains', 'Denmark', 'Copenhagen'],
                       ['/location/location/contains', 'Denmark', 'Reykjavik'],
                       ['/location/country/administrative_divisions', 'Denmark', 'Copenhagen'],
                       ['/location/country/capital', 'Denmark', 'Copenhagen'],
                       ['/location/administrative_division/country', 'Copenhagen', 'Denmark']]


[['/location/location/contains', 'Norway', 'Oslo'],
 ['/location/location/contains', 'Sweden', 'Stockholm'],
 ['/location/country/administrative_divisions', 'Norway', 'Oslo'],
 ['/location/country/capital', 'Norway', 'Oslo'],
 ['/location/administrative_division/country', 'Oslo', 'Norway'],
 ['/location/country/capital', 'Sweden', 'Stockholm'],
 ['/location/location/contains', 'Denmark', 'Copenhagen'],
 ['/location/location/contains', 'Denmark', 'Reykjavik'],
 ['/location/country/administrative_divisions', 'Denmark', 'Copenhagen'],
 ['/location/country/capital', 'Denmark', 'Copenhagen'],
 ['/location/administrative_division/country', 'Copenhagen', 'Denmark']]